In [1]:
#
import sys

# from devolucao import get_df_devol
sys.path.append("..")
import DB
import workdays
import datetime
import pandas as pd
import numpy as np
import carteira_ibov
import taxas
import os
pd.options.mode.chained_assignment = None  # default='warn'
import config
import psycopg2
import pandas as pd
import workdays
import pyodbc
#
holidays_br = workdays.load_holidays("BR")
holidays_b3 = workdays.load_holidays("B3")

dt = datetime.date.today()
vcto_0 = dt
dt_pos = workdays.workday(dt, -1, holidays_br)


dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt_3 = workdays.workday(dt, -3, holidays_b3)
dt_4 = workdays.workday(dt, -4, holidays_b3)

dt_next_1 = workdays.workday(dt, 1, holidays_b3)
vcto_1 = dt_next_1
dt_next_2 = workdays.workday(dt, 2, holidays_b3)
vcto_2 = dt_next_2
dt_next_3 = workdays.workday(dt, 3, holidays_b3)
vcto_3 = dt_next_3
dt_next_4 = workdays.workday(dt, 4, holidays_b3)
vcto_4 = dt_next_4
dt_next_5 = workdays.workday(dt, 5, holidays_b3)
vcto_5 = dt_next_5

dt_liq = workdays.workday(dt_1, 4, holidays_b3)


In [ ]:
DB.get_taxas(1)[['tckrsymb','takravrgrate']].to_clipboard()

In [35]:
db_conn_test = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user=config.DB_TESTE_USER,
password=config.DB_TESTE_PASS)
db_conn_risk = psycopg2.connect(
host=config.DB_RISK_HOST,
dbname=config.DB_RISK_NAME,
user=config.DB_RISK_USER,
password=config.DB_RISK_PASS,
)

query = f"select * from tbl_alugueisconsolidados where dte_data='{dt_1.strftime('%Y-%m-%d')}'"

# query2 = f"""select contrato,cotliq, qtde, liquidacao from st_alugcustcorr where "data"= '{dt_1.strftime('%Y-%m-%d')}' """

query2 =  f"SELECT registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, 'T',vencimento,100*taxa, cotliq, reversor, codigo, st_alugcustcorr.contrato, (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) as saldo \
        from st_alugcustcorr left join st_alug_devolucao on st_alugcustcorr.cliente=st_alug_devolucao.cliente and st_alugcustcorr.contrato=st_alug_devolucao.contrato and dataliq>'{dt_1}' and dataliq<='{dt_liq.strftime('%Y-%m-%d')}' \
        where data='{dt_1.strftime('%Y-%m-%d')}' and qtde>0\
        group by registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, vencimento,taxa,cotliq, reversor, codigo, st_alugcustcorr.contrato\
        HAVING (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) <> 0 \
        UNION SELECT registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, 'D',vencimento,100*taxa, cotliq, reversor, codigo, st_alugcustcorr.contrato, (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) as saldo \
        from st_alugcustcorr left join st_alug_devolucao on st_alugcustcorr.contrato=st_alug_devolucao.contrato and dataliq>'{dt_1.strftime('%Y-%m-%d')}' and dataliq<='{dt_liq.strftime('%Y-%m-%d')}'\
        where data='{dt_1.strftime('%Y-%m-%d')}' and qtde<0\
        group by registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, vencimento,taxa,cotliq, reversor, codigo, st_alugcustcorr.contrato\
        HAVING (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) <> 0 order by codigo,vencimento " 


df = pd.read_sql(query, db_conn_test)
price = pd.read_sql(query2, db_conn_risk)
db_conn_risk.close()
db_conn_test.close()


In [36]:
price

,registro,cliente,corretora,?column?,vencimento,?column?,cotliq,reversor,codigo,contrato,saldo
0,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424541570001-2,-500.0
1,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424539690001-2,-500.0
2,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424541790001-2,-1400.0
3,2023-05-02,KAPITALO KAPPA MASTER FIM,UBS BRASIL CCTVM S/A,D,2023-06-12,32.61,22.77,TD,AALR3,2023050200454612730001-1,-700.0
4,2023-03-22,KAPITALO ZETA MASTER FIM,NECTON INVESTIMENTOS S.A. CVMC,T,2023-05-04,5.75,13.99,TD,ABEV3,2023032200415016710001-2,15000.0
...,...,...,...,...,...,...,...,...,...,...,...
17470,2023-05-03,KAPITALO ZETA MASTER FIM,XP INVESTIMENTOS CCTVM S/A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300372156570001-2,4475.0
17471,2023-05-03,KAPITALO KAPPA MASTER FIM,ORAMA DTVM S A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300455344610001-1,2574.0
17472,2023-05-03,KAPITALO ZETA MASTER FIM,XP INVESTIMENTOS CCTVM S/A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300372156550001-2,6030.0
17473,2023-05-03,KAPITALO ZETA MASTER FIM,NECTON INVESTIMENTOS S.A. CVMC,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300361917990001-2,2000.0


In [42]:
db_conn_risk = psycopg2.connect(
host=config.DB_RISK_HOST,
dbname=config.DB_RISK_NAME,
user=config.DB_RISK_USER,
password=config.DB_RISK_PASS,
)
query2 =  f"select  data, registro, (taxa*100) as taxa ,cotliq , vencimento,corretora, contrato, descoberta as saldo, cliente, codigo from st_alugcustcorr where data='{dt_1.strftime('%Y-%m-%d')}' and descoberta !=0 " 
price = pd.read_sql(query2, db_conn_risk)

db_conn_risk.close()

In [44]:
price = pd.read_excel('imbarq_file.xlsx')

In [45]:
price.rename(columns={'cotliq':'preco','contrato':'str_numcontrato','cliente':'str_fundo'},inplace=True)
filt = price.loc[price['saldo']!=0][['preco','str_numcontrato','saldo']]


In [46]:
filt

,preco,str_numcontrato,saldo
1,1.47,2023041900423934400001-2,-12233
2,10.10,2023041200448361060001-1,9674
5,0.61,2023042600400163290001-2,-16010
17,0.98,2023033000444158040001-1,-17797
26,6.00,2023042000451208820001-1,-2414
...,...,...,...
21507,67.88,2023050300455869090001-1,7969
21508,67.88,2023050350000045510001-2,2573
21509,67.88,2023050350000045510002-2,1922
21510,67.88,2023050350000045510003-2,18


In [47]:
filt['tipo'] = filt['saldo'].apply(lambda x: 'T' if x>0 else 'D')

In [48]:
df.fillna(0,inplace=True)
df = df[(df['str_mesa'].isin(['Kapitalo 11.1','Kapitalo 1.0'])) & (df['dbl_quantidade']!=0)]
df['codigo'] = df['str_serie'].apply(lambda x: x.split('-')[0])
df['tipo'] = df['str_serie'].apply(lambda x: x.split('-')[1])
df = df.merge(filt,on=['str_numcontrato','tipo'],how='inner')
df = df.drop_duplicates()

In [49]:
check = filt.drop_duplicates().merge(df[['tipo','str_numcontrato','dbl_quantidade']].drop_duplicates().groupby(['tipo','str_numcontrato']).agg({'dbl_quantidade':sum}).reset_index().drop_duplicates(),on=['tipo','str_numcontrato'],how='inner')

In [50]:
rebal = check.loc[(check['dbl_quantidade']!=check['saldo'])]


In [51]:
df[df['str_numcontrato']=='2023032800443320550001-1']

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,codigo,tipo,preco,saldo
31,2023-05-03,KAPITALO ZETA MASTER FIA,Emprestimo RV,CRFB,"CRFB3-T-2,64-20230510-BALCAO",Kapitalo 11.1,Bolsa 2,2023032800443320550001-1,33.0,CRFB3,T,11.47,67
32,2023-05-03,KAPITALO ZETA MASTER FIA,Emprestimo RV,CRFB,"CRFB3-T-2,64-20230510-BALCAO",Kapitalo 1.0,Bolsa 2,2023032800443320550001-1,33.0,CRFB3,T,11.47,67


In [52]:
rebal

,preco,str_numcontrato,saldo,tipo,dbl_quantidade
1,0.98,2023032900430014760001-2,2491,T,8809.0
4,2.66,2023033000444231810001-1,10379,T,690.0
6,4.63,2023033000444231740001-1,2100,T,4428.0
7,4.63,2023033000444231740001-1,6528,T,4428.0
8,2.69,2023033000444310030001-1,1282,T,10749.0
9,14.58,2023032900444011290001-1,180434,T,88066.0
10,62.33,2023032800443426110001-1,1400,T,604.0
21,18.92,2023033000444455270001-1,22938,T,7200.0
22,20.12,2023041100447689040001-1,1300,T,5231.0
26,31.16,2023033000444339430001-1,1004,T,15604.0


In [53]:
ex_df = df[df['str_numcontrato'].isin(rebal['str_numcontrato'])]

In [54]:
f_df = df[~df['str_numcontrato'].isin(rebal['str_numcontrato'])]

In [55]:
ex_df = ex_df.merge(rebal[['str_numcontrato', 'tipo', 'dbl_quantidade']].rename(columns={'dbl_quantidade':'full size'}),on=['str_numcontrato', 'tipo'],how='inner')

In [56]:
ex_df

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,codigo,tipo,preco,saldo,full size
0,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431486510001-2,2990.0,YDUQ3,T,7.17,5173,13558.0
1,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 1.0,Bolsa 2,2023033100431486510001-2,10568.0,YDUQ3,T,7.17,5173,13558.0
2,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,TAEE,"TAEE11-T-0,05-20230516-BALCAO",Kapitalo 11.1,Bolsa 2,2023040300445399840001-1,5209.0,TAEE11,T,34.87,301,5209.0
3,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,RAIZ,"RAIZ4-T-0,1-20230516-BALCAO",Kapitalo 1.0,Bolsa 2,2023040300445151740001-1,1000.0,RAIZ4,T,2.86,4400,1000.0
4,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,LJQQ,"LJQQ3-T-4,13-20230522-BALCAO",Kapitalo 11.1,Bolsa 2,2023041000447360570001-1,4.0,LJQQ3,T,3.66,4027,241.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,LWSA,"LWSA3-T-11,13-20230512-BALCAO",Kapitalo 11.1,Bolsa 2,2023033000444231740001-1,4428.0,LWSA3,T,4.63,6528,4428.0
72,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,LWSA,"LWSA3-T-11,13-20230512-BALCAO",Kapitalo 11.1,Bolsa 2,2023033000444231740001-1,4428.0,LWSA3,T,4.63,6528,4428.0
73,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,HAPV,"HAPV3-T-0,93-20230512-BALCAO",Kapitalo 11.1,Bolsa 2,2023033000444231810001-1,690.0,HAPV3,T,2.66,10379,690.0
74,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CASH,"CASH3-T-4,23-20230511-BALCAO",Kapitalo 11.1,Bolsa 2,2023032900430014760001-2,8809.0,CASH3,T,0.98,2491,8809.0


In [57]:
ex_df['prop'] = ex_df['dbl_quantidade']/ex_df['full size']

ex_df['new_qtd'] = round(ex_df['prop']*ex_df['saldo'])

In [58]:
ex_df['dbl_quantidade'] = ex_df['new_qtd']

In [59]:
ex_df = ex_df[f_df.columns]

In [60]:
df = pd.concat([ex_df,f_df])

In [61]:
df

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,codigo,tipo,preco,saldo
0,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431486510001-2,1141.0,YDUQ3,T,7.17,5173
1,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 1.0,Bolsa 2,2023033100431486510001-2,4032.0,YDUQ3,T,7.17,5173
2,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,TAEE,"TAEE11-T-0,05-20230516-BALCAO",Kapitalo 11.1,Bolsa 2,2023040300445399840001-1,301.0,TAEE11,T,34.87,301
3,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,RAIZ,"RAIZ4-T-0,1-20230516-BALCAO",Kapitalo 1.0,Bolsa 2,2023040300445151740001-1,4400.0,RAIZ4,T,2.86,4400
4,2023-05-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,LJQQ,"LJQQ3-T-4,13-20230522-BALCAO",Kapitalo 11.1,Bolsa 2,2023041000447360570001-1,67.0,LJQQ3,T,3.66,4027
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CRFB,"CRFB3-T-0,94-20230601-BALCAO",Kapitalo 11.1,Bolsa 2,2023042000451208880001-1,652.0,CRFB3,T,10.22,652
222,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,BRKM,"BRKM5-T-0,1-20230601-BALCAO",Kapitalo 11.1,Bolsa 2,2023042000451208930001-1,30.0,BRKM5,T,19.70,30
223,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,BEEF,"BEEF3-T-0,3-20230601-BALCAO",Kapitalo 11.1,Bolsa 2,2023042000451208660001-1,1970.0,BEEF3,T,9.21,1970
224,2023-05-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,ASAI,"ASAI3-T-5,05-20230601-BALCAO",Kapitalo 11.1,Bolsa 2,2023042000451208590001-1,1362.0,ASAI3,T,12.87,1362


In [ ]:
# df['dbl_quantidade'] = df['saldo']

df['taxa'] = df['str_serie'].apply(lambda x: x.split('-')[2].replace(',','.'))

df['modalidade'] = df['str_serie'].apply(lambda x: x.split('-')[4])
df['vencimento'] = df['str_serie'].apply(lambda x: x.split('-')[3])
df['volume'] = df['dbl_quantidade'].astype(float)*df['taxa'].astype(float)
df = df.drop_duplicates()
ctos_btc = df
vencimentos = df



In [ ]:
df

In [ ]:
df = df.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum,'volume':sum}).reset_index()

In [ ]:
df['taxa media'] = (df['volume']/df['dbl_quantidade']).apply(lambda x: round(x,2))

In [ ]:
btc = pd.pivot_table(df,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns=['tipo'],values=['dbl_quantidade','taxa media']).reset_index().fillna(0)

In [ ]:
btc.columns = ['fundo',	'mesa',	'str_estrategia','codigo','DOADO','TOMADO','TAXA DOADORA','TAXA TOMADORA']

In [ ]:
import workdays
holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()


dt_1 = workdays.workday(dt, -1, holidays_b3)

dt_1 = workdays.workday(dt, -1, holidays_b3)
vcto_0 = dt.strftime('%d/%m/%Y')
dt_pos = workdays.workday(dt, -1, holidays_br)
venc_interna = workdays.workday(dt, 1, holidays_br)



df_pos = DB.get_equity_positions_mesas(None,dt_1)


df_pos = pd.DataFrame(df_pos[['str_fundo','str_mesa','str_estrategia',"regexp_replace", "sum"]])
df_pos.rename(columns={"regexp_replace": "codigo", "sum": "position","str_fundo":"fundo",'str_mesa':'mesa'}, inplace=True)


df_pos = df_pos[ (df_pos['mesa'].isin(['Kapitalo 11.1','Kapitalo 1.0'])) & ~(df_pos['fundo'].isin(['KAPITALO CLASS B', 'KAPITALO CLASS K', 'KAPITALO CLASS OMEGA']))]

In [ ]:
mapa = df_pos.merge(btc,on=['fundo','mesa','str_estrategia','codigo'],how='outer').fillna(0)

# Incluindo Internas K11

In [ ]:
# import pandas as pd
# k11 = pd.read_excel('internas.xlsx')

In [ ]:
# k11['codigo'] = k11['SERIE'].apply(lambda x: x.split('-')[0])
# k11['taxa'] = k11['SERIE'].apply(lambda x: x.split('-')[2].replace(',','.'))
# k11['tipo'] = k11['SERIE'].apply(lambda x: x.split('-')[1])
# k11['modalidade'] = k11['SERIE'].apply(lambda x: x.split('-')[4])
# k11['vencimento'] = k11['SERIE'].apply(lambda x: x.split('-')[3])
# k11['volume'] = k11['NOTIONAL'].astype(float)*k11['taxa'].astype(float)

In [ ]:


# k11 = k11.rename(columns={'ALOCACAO':'str_fundo','MESA':'str_mesa','ESTRATEGIA':'str_estrategia','NOTIONAL':'dbl_quantidade'})

In [ ]:
# k11 = k11.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum,'volume':sum}).reset_index()

In [ ]:
# k11 = pd.pivot_table(k11,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns='tipo',values='dbl_quantidade').reset_index().fillna(0)

In [ ]:
# k11.rename(columns={'D':'interna DOADO','T':'interna TOMADO'},inplace=True)

In [ ]:
# k11.columns = ['fundo',	'mesa',	'str_estrategia','codigo','interna DOADO','interna TOMADO']

# Incluindo Internas Mesas

In [ ]:
mesa = pd.read_excel('mesa.xlsx')

In [ ]:
mesa = pd.pivot_table(mesa,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns='tipo',values='lote estrategia').reset_index()

In [ ]:
# mesa['mesas DOADO'] = mesa['D']*-1
mesa['mesas DOADO'] = 0

mesa['mesas TOMADO'] = mesa['T']*-1

In [ ]:
mesa = mesa[['str_fundo','str_mesa','str_estrategia','codigo','mesas DOADO','mesas TOMADO']].fillna(0)

In [ ]:
mesa.columns = ['fundo','mesa',	'str_estrategia','codigo','mesas DOADO','mesas TOMADO']

# Internas sinteticas + mapa

In [ ]:
# mapa = mapa.merge(k11,on=['fundo',	'mesa',	'str_estrategia','codigo'],how='outer').merge(mesa,on=['fundo',	'mesa',	'str_estrategia','codigo'],how='outer').fillna(0)

In [ ]:
mapa = mapa.merge(mesa,on=['fundo',	'mesa',	'str_estrategia','codigo'],how='outer').fillna(0)

# Vencimentos

In [ ]:

vencimentos['vencimento'] = vencimentos['vencimento'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d').date())

In [ ]:
aux  = vencimentos.loc[(vencimentos['vencimento']>= dt) & (vencimentos['vencimento']< dt_next_5)]

In [ ]:
aux = aux.groupby(['str_fundo','codigo','str_mesa','str_estrategia','vencimento']).agg({'dbl_quantidade':sum}).reset_index()

In [ ]:
venc = pd.pivot_table(aux,index=['str_fundo','codigo','str_mesa','str_estrategia'],columns='vencimento',values='dbl_quantidade').reset_index().fillna(0)

In [ ]:
venc.rename(columns={'str_fundo':'fundo','str_mesa':'mesa'},inplace=True)

In [ ]:
mapa = mapa.merge(venc,on=['fundo','codigo','mesa','str_estrategia'],how='outer').fillna(0)

# Movimentações

In [ ]:
def get_equity_trades():
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)    
    query=f"SELECT * FROM tbl_auxboletas1 where dte_data > '{dt_3.strftime('%Y-%m-%d')}' and ((str_mesa='Kapitalo 11.1' and str_mercado='Acao') or (str_mesa='Kapitalo 1.0' and  str_estrategia = 'Bolsa 2' and str_mercado='Acao') ) and str_fundo not in ('KAPITALO CLASS B', 'KAPITALO CLASS OMEGA')" 
    try:
        df =pd.read_sql(query,db_conn_test)
    except:
        pass
    db_conn_test.close()

    return df

In [ ]:
trades = get_equity_trades()

In [ ]:
trades  = trades.groupby(['str_fundo','str_serie','str_mesa','str_estrategia','dte_data']).agg({'dbl_lote':sum}).reset_index()

In [ ]:
trades = pd.pivot_table(trades,index=['str_fundo','str_serie','str_mesa','str_estrategia'],columns='dte_data',values='dbl_lote').reset_index()


In [ ]:
trades['str_serie'] = trades['str_serie'].apply(lambda x: x.replace(' BZ EQUITY',''))
trades.rename(columns={
    dt_2:'mov_0',
    dt_1:'mov_1',
    'str_fundo':'fundo',
    'str_mesa':'mesa',
    'str_serie':'codigo'
    },inplace=True
)

In [ ]:
mapa = mapa.merge(trades,on=['fundo','codigo','mesa','str_estrategia'],how='outer').fillna(0)

In [ ]:
dt

In [ ]:
query_ibotz = f" select str_fundo,str_mesa,str_estrategia,str_codigo,str_serie, str_numcontrato,sum(dbl_quantidade) as dbl_quantidade  from ibotz.tbl_boletasalugueis_ibotz where dte_data='{dt.strftime('%Y-%m-%d')}' and str_mesa in ('Kapitalo 11.1','Kapitalo 1.0') and str_mercado like '%Emprestimo RV%' group by str_fundo,str_mesa,str_estrategia,str_codigo,str_serie, str_numcontrato"
db_conn_test = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user=config.DB_TESTE_USER,
password=config.DB_TESTE_PASS)

btc_ibotz = pd.read_sql(query_ibotz,db_conn_test)
db_conn_test.close()

In [ ]:
btc_ibotz['codigo'] = btc_ibotz['str_serie'].apply(lambda x: x.split('-')[0])
btc_ibotz['tipo'] = btc_ibotz['str_serie'].apply(lambda x: x.split('-')[1])
btc_ibotz['modalidade'] = btc_ibotz['str_serie'].apply(lambda x: x.split('-')[-1])
# btc_ibotz = btc_ibotz[btc_ibotz['modalidade']!='INTERNO']
btc_ibotz = btc_ibotz.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum}).reset_index()


In [ ]:
btc_ibotz

In [ ]:
btc_ibotz = pd.pivot_table(btc_ibotz,columns='tipo',index=['str_fundo','str_mesa','codigo','str_estrategia'],values='dbl_quantidade').reset_index().fillna(0).rename(
    columns=
    {'str_fundo':'fundo',
    'str_mesa':'mesa',
    'D':'trade_doado',
    'T':'trade_tomado'}
    )

In [ ]:
from BBI import get_bbi
recalls =  get_bbi.req_mov_alugueis_solicitacao_liq(dt)

In [ ]:
recalls[(recalls['codneg']=='ARZZ3')]

In [ ]:
recalls = recalls.rename(columns={'contrato':'str_numcontrato','Fundo_Kptl':'str_fundo','codneg':'codigo'}).merge(ctos_btc[['str_fundo','str_numcontrato','codigo','str_mesa','str_estrategia']],on=['str_fundo','str_numcontrato','codigo'],how='inner')

In [ ]:
recalls['qtde'] = recalls.apply(lambda row: -abs(row['qtde']) if row['tipo']=='TOMADOR' else abs(row['qtde']),axis=1)


In [ ]:
recalls['dvd'] = recalls['str_numcontrato'].apply(lambda x: recalls['str_numcontrato'].tolist().count(x))

In [ ]:
recalls = pd.pivot_table(recalls,columns='datalimite',index=['str_fundo','codigo','str_mesa','str_estrategia','dvd'],values='qtde').reset_index().fillna(0)

In [ ]:
recalls.columns[-1].to_pydatetime().date()

In [ ]:
recalls  = recalls.rename(columns = {pd.Timestamp(dt_next_3):'PendRecallD3',pd.Timestamp(dt_next_2):'PendRecallD2',pd.Timestamp(dt_next_1):'PendRecallD1'})

In [ ]:
import math
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

In [ ]:
recalls

In [ ]:
for i in range(1,4):
    if f"PendRecallD{i}" not in recalls.columns:
        recalls[f"PendRecallD{i}"] = 0





In [ ]:

recalls['PendRecallD1'] = recalls.apply( lambda row: round_up(float(row['PendRecallD1']/row['dvd'])),axis=1 )
recalls['PendRecallD2'] = recalls.apply( lambda row: round_up(float(row['PendRecallD2']/row['dvd'])),axis=1 )
recalls['PendRecallD3'] = recalls.apply( lambda row: round_up(float(row['PendRecallD3']/row['dvd'])),axis=1 )

In [ ]:
mapa = mapa.merge(recalls.rename(columns = {'str_fundo':'fundo','str_mesa':'mesa'}),on=['fundo','mesa', 'str_estrategia','codigo'],how='left').fillna(0)

In [ ]:
mapa = mapa.merge(btc_ibotz,on=['fundo','mesa', 'str_estrategia','codigo'],how='outer').fillna(0)

In [ ]:
# mapa["PendRecallD1"] = 0
# mapa["PendRecallD2"] = 0
# mapa["PendRecallD3"] = 0
mapa['mov_2'] = 0

In [ ]:
if not 'trade_doado' in mapa.columns:
    mapa['trade_doado'] = 0 
if not 'trade_tomado' in mapa.columns:
    mapa['trade_tomado'] = 0


if not 'interna DOADO' in mapa.columns:
    mapa['interna DOADO'] = 0 
if not 'interna TOMADO' in mapa.columns:
    mapa['interna TOMADO'] = 0

In [ ]:

mapa["pos_doada"] = mapa["DOADO"] + mapa["trade_doado"] + mapa['interna DOADO'] +  mapa['mesas DOADO']
mapa["pos_tomada"] = mapa["TOMADO"] + mapa["trade_tomado"] + mapa['interna TOMADO'] + mapa['mesas TOMADO']
mapa["net_alugado"] = mapa["pos_doada"] + mapa["pos_tomada"]
mapa["custodia_aux"] = mapa["position"] + mapa["net_alugado"] - mapa["mov_0"] - mapa["mov_1"]

# mapa['custodia_janela'] = Saldo doador - Mov_0 (dia atual)
try:
    mapa["custodia_0"] = mapa["custodia_aux"] - mapa[vcto_0] + mapa["mov_0"] 
except:
    mapa["custodia_0"] = mapa["custodia_aux"] + mapa["mov_0"] 
mapa["custodia_0"].fillna(0, inplace=True)

try:
    mapa["custodia_1"] = mapa["custodia_0"] + mapa["mov_1"] - mapa[vcto_1] + mapa["PendRecallD1"]
except:
    mapa["custodia_1"] = mapa["custodia_0"] + mapa["mov_1"] + mapa["PendRecallD1"]
    
try:
    mapa["custodia_2"] = mapa["custodia_1"] - mapa[vcto_2] + mapa["PendRecallD2"] + mapa["mov_2"]
except:
    mapa["custodia_2"] = mapa["custodia_1"] + mapa["PendRecallD2"] + mapa["mov_2"]
    
mapa["custodia_3"] = mapa["custodia_2"] - mapa[vcto_3] + mapa["PendRecallD3"]

mapa["to_borrow_0"] = np.minimum(0, mapa["custodia_0"])
mapa["to_borrow_0"].fillna(0, inplace=True)
mapa["to_borrow_1"] = np.minimum(0, mapa["custodia_1"] - mapa["to_borrow_0"])
mapa["to_borrow_1"].fillna(0, inplace=True)
mapa["to_borrow_2"] = np.minimum(
    0, mapa["custodia_2"] - mapa["to_borrow_0"] - mapa["to_borrow_1"]
)
mapa["to_borrow_2"].fillna(0, inplace=True)
mapa["to_borrow_3"] = np.minimum(
    0, mapa["custodia_3"] - mapa["to_borrow_0"] - mapa["to_borrow_1"] - mapa["to_borrow_2"]
)

mapa["to_borrow_3"].fillna(0, inplace=True)



In [ ]:
mapa["custodia_exaluguel"] = (
    mapa["custodia_aux"]
    - mapa["net_alugado"]
    + np.minimum.reduce(
        [
            mapa["mov_0"],
            mapa["mov_0"] + mapa["mov_1"],
            mapa["mov_0"] + mapa["mov_1"] + mapa["mov_2"],
        ]
    )
)
mapa["devol_tomador_of"] = np.minimum(
    -np.minimum(mapa["custodia_exaluguel"], 0) - mapa["pos_tomada"], 0
)

mapa["devol_tomador"] = np.minimum(
    -np.minimum(mapa["custodia_aux"], 0) - mapa["pos_tomada"], 0
)
mapa.loc[mapa['to_borrow_0']<0,'devol_tomador'] = 0

mapa["devol_doador"] = np.maximum(
    -np.maximum(mapa["custodia_exaluguel"], 0)
    - mapa["pos_doada"]
    - mapa["PendRecallD1"]
    - mapa["PendRecallD2"]
    - mapa["PendRecallD3"],
    0,
)
mapa["devol_tomador"].fillna(0, inplace=True)
mapa["devol_doador"].fillna(0, inplace=True)



In [ ]:
mapa["to_lend"] = mapa.apply(
    lambda row: 0
    if (row["custodia_exaluguel"] + row["pos_doada"] < 0)
    else row["custodia_exaluguel"] + row["pos_doada"]
    if row["custodia_exaluguel"] > 0
    else 0,
    axis=1,
)


In [ ]:
mapa

# Ajuste de contratos

In [ ]:
ind_devol =mapa[mapa['devol_tomador']!=0][['fundo','mesa','str_estrategia','codigo','custodia_0','devol_tomador']]
ajuste = mapa[mapa['to_borrow_0']!=0][['fundo','mesa','str_estrategia','codigo','to_borrow_0']]
sinal_ajuste = ajuste[['fundo','codigo','to_borrow_0']]
ind_devol['devol_tomador'] = -ind_devol['devol_tomador']
ind_devol = ind_devol.merge(sinal_ajuste,on=['fundo','codigo'],how='inner').drop_duplicates()

ind_devol['to_borrow_0'] = -ind_devol['to_borrow_0']
ind_devol = ind_devol.groupby(['fundo', 'mesa', 'str_estrategia', 'codigo', 'custodia_0','devol_tomador']).agg({'to_borrow_0':sum}).reset_index()
ind_devol['indicativo devol'] = ind_devol.apply(lambda row: min(row['to_borrow_0'],max(row['custodia_0'],row['devol_tomador'])),axis=1)

ind_devol = ind_devol.drop_duplicates()
ctos_btc.rename(columns={'str_fundo':'fundo','str_mesa':'mesa'},inplace=True)
devol = ctos_btc.merge(ind_devol, on=["fundo",'mesa','str_estrategia',"codigo"], how="inner")
devol = devol.drop_duplicates()
devol = devol.loc[(devol['tipo']=='T') & (devol['dbl_quantidade']>0)]

devol = devol.sort_values('codigo')
devol['Devolucao']=0
ind_devol = ind_devol.groupby(['fundo', 'mesa', 'str_estrategia', 'codigo']).agg({'indicativo devol':sum}).reset_index()


In [ ]:
devol = devol[['dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
       'str_estrategia', 'str_numcontrato', 'dbl_quantidade', 'preco',
       'codigo','Devolucao']].drop_duplicates()

In [ ]:
for i, row in devol.iterrows():
        saldo = ind_devol.loc[(ind_devol['fundo']==row['fundo'])&(ind_devol['codigo']==row['codigo'])&(ind_devol['mesa']==row['mesa'])&(ind_devol['str_estrategia']==row['str_estrategia']),'indicativo devol'].sum()
        devol.loc[(devol['fundo']==row['fundo'])&(devol['codigo']==row['codigo'])&(devol['mesa']==row['mesa'])& (devol['str_estrategia']==row['str_estrategia'] )& (devol['str_numcontrato']==row['str_numcontrato']),'Devolucao'] = min(row['dbl_quantidade'],saldo)
        ind_devol.loc[(ind_devol['fundo']==row['fundo'])&(ind_devol['codigo']==row['codigo'])&(ind_devol['mesa']==row['mesa'])&(ind_devol['str_estrategia']==row['str_estrategia']),'indicativo devol'] = round( saldo -min(row['dbl_quantidade'],saldo) ,2)

In [ ]:
# mapa.

In [ ]:
rebal_tomadores = devol[devol['Devolucao']!=0]

In [ ]:
## Preciso realizar com contratos doadores
ind_devol[ind_devol['indicativo devol']!=0]

In [ ]:
alloc_tomadores = rebal_tomadores

In [ ]:
# alloc_tomadores['dbl_quantidade'] = alloc_tomadores['Devolucao'] 

In [ ]:
filt_map = mapa[mapa['custodia_0']>0][['fundo','mesa','str_estrategia','codigo']]

In [ ]:
alloc_tomadores = alloc_tomadores.merge(filt_map,on=['fundo','mesa','str_estrategia','codigo'],how='inner')

In [ ]:
alloc_tomadores.rename(columns={'Devolucao':'Alocacao'},inplace= True)

In [ ]:
alloc_tomadores['estrategia target'] = 0

In [ ]:
map_ajuste = ajuste

In [ ]:
map_ajuste['to_borrow_0'] = -map_ajuste['to_borrow_0']

In [ ]:
map_ajuste

In [ ]:
def qtd_dist(row):

    if abs(row)>1000000:
        if row>0:
            return [100000 for i in range(abs(int(row/100000)))]
        else:
            return [-100000 for i in range(abs(int(row/100000)))]
    
    if abs(row)>100000:
        if row>0:
            return [10000 for i in range(abs(int(row/10000)))]
        else:
            return [-10000 for i in range(abs(int(row/10000)))]
    if abs(row)>10000:
        if row>0:
            return [1000 for i in range(abs(int(row/1000)))]
        else:
            return [-1000 for i in range(abs(int(row/1000)))]
    elif abs(row)>1000:
        if row>0:
            return [100 for i in range(abs(int(row/100)))]
        else:
            return [-100 for i in range(abs(int(row/100)))]
    else:
        if row>0:
            return [1 for i in range(abs(int(row)))]
        else:
            return [-1 for i in range(abs(int(row)))]



In [ ]:
def ult_dist(row):
    if row>0:
        return [1 for i in range(abs(int(row)))]
    else:
        return [-1 for i in range(abs(int(row)))]

In [ ]:
def dist_ctos(alloc_tomadores):
       

       alloc_tomadores["quebra"] = alloc_tomadores["dbl_quantidade"].apply(lambda x: qtd_dist(x))
       alloc_tomadores = alloc_tomadores.explode("quebra")
       # print(alloc_tomadores.columns)
       dif = alloc_tomadores.groupby(['dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
              'str_estrategia', 'str_numcontrato', 'dbl_quantidade', 'preco',
              'codigo', 'Alocacao', 'estrategia target']).agg({'quebra':sum}).reset_index().drop_duplicates()
       dif['diff'] = dif['dbl_quantidade'] - dif['quebra']
       dif = dif[dif['diff']!=0]
       dif['dbl_quantidade'] = dif['diff']
       dif["quebra"] = dif["dbl_quantidade"].apply(lambda x: ult_dist(x))
       dif = dif.explode("quebra")
       alloc_tomadores = pd.concat([alloc_tomadores,dif])
       alloc_tomadores['aux'] = alloc_tomadores['dbl_quantidade'] 
       alloc_tomadores['dbl_quantidade'] = alloc_tomadores['quebra']

       return alloc_tomadores

In [ ]:
alloc_tomadores = dist_ctos(alloc_tomadores=alloc_tomadores)

In [ ]:
alloc_tomadores = alloc_tomadores.sort_values(by=['quebra'], ascending=False)

In [ ]:
map_ajuste

In [ ]:
alloc_tomadores.index = [x+1 for x in range(alloc_tomadores.shape[0])]
map_ajuste.index = [x+1 for x in range(map_ajuste.shape[0])]

In [ ]:
alloc_tomadores['volta'] = 0
alloc_tomadores['estrategia target'] = 0
alloc_tomadores['mesa target'] = 0
from tqdm import tqdm
aux = pd.DataFrame()
ativos_concluidos = []
with tqdm(total=map_ajuste.shape[0]) as pbar:
    for i, row in (map_ajuste.iterrows()):
        pbar.update(1)
        saldo = row['to_borrow_0']
        aux = alloc_tomadores[alloc_tomadores['codigo']==row['codigo']]
        
        for x, x_row in aux.iterrows():
            if int(saldo) ==0:
                break
            if (x_row['codigo']==row['codigo']) & (x_row['fundo']==row['fundo']):
                if (alloc_tomadores.loc[x,'estrategia target']==0):
                    total = min(saldo,x_row['dbl_quantidade'])
                    alloc_tomadores.loc[x,'volta'] = total
                    alloc_tomadores.loc[x,'mesa target'] = row['mesa']
                    alloc_tomadores.loc[x,'estrategia target'] = row['str_estrategia']
                    saldo = saldo - total
                    map_ajuste.loc[i,'to_borrow_0']  = saldo
            

In [ ]:
map_ajuste

In [ ]:
alloc_tomadores['saldo'] = alloc_tomadores['quebra'] - alloc_tomadores['volta']

In [ ]:
saldo_alloc_tomadores = alloc_tomadores[alloc_tomadores['saldo']!=0]

In [ ]:
saldo_alloc_tomadores['dbl_quantidade'] = saldo_alloc_tomadores['saldo']

In [ ]:
saldo_alloc_tomadores["quebra"] = saldo_alloc_tomadores["dbl_quantidade"].apply(lambda x: ult_dist(x))
saldo_alloc_tomadores = saldo_alloc_tomadores.explode("quebra")

In [ ]:
saldo_alloc_tomadores['dbl_quantidade'] = saldo_alloc_tomadores['quebra']

In [ ]:
map_ajuste = map_ajuste[map_ajuste['to_borrow_0']!=0]

In [ ]:
saldo_alloc_tomadores.index = [x+1 for x in range(saldo_alloc_tomadores.shape[0])]
map_ajuste.index = [x+1 for x in range(map_ajuste.shape[0])]

In [ ]:
saldo_alloc_tomadores['volta'] = 0
saldo_alloc_tomadores['estrategia target'] = 0
saldo_alloc_tomadores['mesa target'] = 0
from tqdm import tqdm
aux = pd.DataFrame()
ativos_concluidos = []
with tqdm(total=map_ajuste.shape[0]) as pbar:
    for i, row in (map_ajuste.iterrows()):
        pbar.update(1)
        saldo = row['to_borrow_0']
        aux = saldo_alloc_tomadores[saldo_alloc_tomadores['codigo']==row['codigo']]
        total = 0
        for x, x_row in aux.iterrows():
            if int(saldo) ==0:
                break
            if (x_row['codigo']==row['codigo']) & (x_row['fundo']==row['fundo']):
                if (saldo_alloc_tomadores.loc[x,'estrategia target']==0):
                    # if ((mapa.loc[(mapa['fundo']==row['fundo'])&(mapa['codigo']==row['codigo'])&(mapa['str_estrategia']==row['str_estrategia'])&(mapa['mesa']==row['mesa']),'custodia_0'].item()) - total ) > 0 :
                        # total = min(saldo,x_row['dbl_quantidade']) + total
                    saldo_alloc_tomadores.loc[x,'volta'] = min(saldo,x_row['dbl_quantidade'])
                    saldo_alloc_tomadores.loc[x,'mesa target'] = row['mesa']
                    saldo_alloc_tomadores.loc[x,'estrategia target'] = row['str_estrategia']
                    saldo = saldo - min(saldo,x_row['dbl_quantidade'])
                    map_ajuste.loc[i,'to_borrow_0']  = saldo
                    # else:
                    #     break

In [ ]:
# saldo_tomadores = alloc_tomadores[alloc_tomadores['estrategia target']==0]
# alloc_tomadores =  alloc_tomadores[alloc_tomadores['estrategia target']!=0]
# espelho= alloc_tomadores.groupby(['fundo','codigo','estrategia target']).agg({'volta':sum}).reset_index()

In [ ]:
map_ajuste = map_ajuste[map_ajuste['to_borrow_0']!=0] 

In [ ]:
map_ajuste

## Rebalanceamento com devolução Doadora

In [ ]:
def dist_ctos(alloc_tomadores):
       

       alloc_tomadores["quebra"] = alloc_tomadores["dbl_quantidade"].apply(lambda x: qtd_dist(x))
       alloc_tomadores = alloc_tomadores.explode("quebra")
       dif = alloc_tomadores.groupby(['dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
              'str_estrategia', 'str_numcontrato', 'dbl_quantidade', 'preco',
              'codigo', 'Alocacao', 'estrategia target']).agg({'quebra':sum}).reset_index().drop_duplicates()
       dif['diff'] = dif['dbl_quantidade'] - dif['quebra']
       dif = dif[dif['diff']!=0]
       dif['dbl_quantidade'] = dif['diff']
       dif["quebra"] = dif["dbl_quantidade"].apply(lambda x: ult_dist(x))
       dif = dif.explode("quebra")
       alloc_tomadores = pd.concat([alloc_tomadores,dif])
       alloc_tomadores['aux'] = alloc_tomadores['dbl_quantidade'] 
       alloc_tomadores['dbl_quantidade'] = alloc_tomadores['quebra']

       return alloc_tomadores

In [ ]:
ctos_btc_d = ctos_btc[ctos_btc['tipo']=='D']
devol_d = ctos_btc_d.merge(map_ajuste, on=["fundo",'mesa','str_estrategia',"codigo"], how="inner")
devol_d['to_borrow_0'] = devol_d['to_borrow_0'].fillna(0)
devol_d = devol_d[devol_d['to_borrow_0']!=0]
devol_d['dbl_quantidade'] = abs(devol_d['dbl_quantidade'])
devol_d['Alocacao'] = 0
devol_d['estrategia target'] = 0
devol_d = dist_ctos(alloc_tomadores=devol_d)
devol_d.index = [x+1 for x in range(devol_d.shape[0])]
map_ajuste.index = [x+1 for x in range(map_ajuste.shape[0])]

In [ ]:
devol_d['volta'] = 0

devol_d['estrategia target'] = 0

devol_d['mesa target'] = 0


with tqdm(total=map_ajuste.shape[0]) as pbar:
    for i, row in (map_ajuste.iterrows()):
        pbar.update(1)
        saldo = row['to_borrow_0']
        for x, x_row in devol_d.iterrows():
            if int(saldo) ==0:
                break
            if (x_row['codigo']==row['codigo']) & (x_row['fundo']==row['fundo']):
                if (devol_d.loc[x,'estrategia target']==0):
                    devol_d.loc[x,'volta'] = min(saldo,x_row['dbl_quantidade'])
                    devol_d.loc[x,'mesa target'] = row['mesa']
                    devol_d.loc[x,'estrategia target'] = row['str_estrategia']
                    saldo = saldo - min(saldo,x_row['dbl_quantidade'])
                    map_ajuste.loc[i,'to_borrow_0']  = saldo
            

In [ ]:
devol_d

In [ ]:
map_ajuste[map_ajuste['to_borrow_0']!=0]

In [ ]:
mapa

# Organizando Boletas

In [ ]:
a = alloc_tomadores.groupby(
    [
    'dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
       'str_estrategia', 'str_numcontrato',
       'codigo','estrategia target', 'mesa target','Alocacao',
    ]
).agg({'volta':sum}).reset_index()



In [ ]:
saldo_alloc_tomadores = saldo_alloc_tomadores.groupby(
    [
    'dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
       'str_estrategia', 'str_numcontrato', 'dbl_quantidade',
       'codigo','estrategia target', 'mesa target','Alocacao',
       
    ]
).agg({'volta':sum}).reset_index()


In [ ]:
saldo_alloc_tomadores = saldo_alloc_tomadores[saldo_alloc_tomadores['estrategia target']!=0]

In [ ]:
devol = devol_d.groupby(
    [
    'dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
       'str_estrategia', 'str_numcontrato', 'dbl_quantidade',
       'codigo','estrategia target', 'mesa target','Alocacao',
       
    ]
).agg({'volta':sum}).reset_index()

In [ ]:
devol = devol[devol['estrategia target']!=0]

In [ ]:
mapa

# Montando boletas

In [ ]:
tomador = pd.concat([a,saldo_alloc_tomadores])

In [ ]:
tomador_g = tomador[['fundo','mesa','str_estrategia','mesa target','estrategia target','codigo','str_serie','volta','str_numcontrato']]

In [ ]:
tomador_g = tomador_g[tomador_g['volta']!=0]
tomador_in = tomador_g[['fundo','mesa target','estrategia target','codigo','str_serie','volta','str_numcontrato']]
tomador_out = tomador_g[['fundo','mesa','str_estrategia','codigo','str_serie','volta','str_numcontrato']]
tomador_out['dbl_quantidade'] = -abs(tomador_out['volta'])
tomador_in['dbl_quantidade'] = abs(tomador_in['volta'])

tomador_in = tomador_in[['fundo','mesa target','estrategia target','codigo','str_serie','dbl_quantidade','str_numcontrato']]
tomador_out = tomador_out[['fundo','mesa','str_estrategia','codigo','str_serie','dbl_quantidade','str_numcontrato']]


In [ ]:
tomador_in

In [ ]:
columns = [
    'str_fundo',
    'str_mesa',
    'str_estrategia',
    'str_codigo', 
    'str_serie',
    'dbl_quantidade',
    'str_numcontrato'
]

In [ ]:
tomador_in.columns = columns
tomador_out.columns = columns



In [ ]:
tomador_geral = pd.concat([tomador_in,tomador_out])

In [ ]:
tomador_geral['str_corretora'] = 'Interna'
tomador_geral['str_clearing'] = 'Interna'
tomador_geral['str_mercado'] = 'Emprestimo RV/AjustePosicao'

In [ ]:
tomador_geral['str_codigo'] = tomador_geral['str_serie'].apply(lambda x: x[0:4])

In [ ]:
tomador_geral.groupby('str_numcontrato').agg({'dbl_quantidade':sum})['dbl_quantidade'].sum()

In [ ]:
teste = tomador_geral

In [ ]:
teste = teste[[
    'str_fundo',
    'str_mesa',
    'str_estrategia',
    'str_corretora',
    'str_clearing',
    'str_mercado',
    'str_codigo', 
    'str_serie',
    'dbl_quantidade',
    'str_numcontrato']]

In [ ]:
teste.to_clipboard()

In [ ]:
# check = ctos_btc.merge(teste.rename(columns={'dbl_quantidade':'trade','str_fundo':'fundo','str_mesa':'mesa'}),on=['fundo',
#     'mesa',
#     'str_estrategia','str_numcontrato'],how='inner')

In [ ]:
# check['new lote'] = check['trade'] + check['dbl_quantidade']


In [ ]:
teste

In [ ]:
devol_out = devol[['fundo','mesa','str_estrategia','codigo','str_serie','volta','str_numcontrato']]
devol_out['dbl_quantidade'] = abs(devol_out['volta'])



In [ ]:
devol_out

In [ ]:
devol_in = pd.DataFrame()

In [ ]:
devol_in['fundo'] = devol_out['fundo']

devol_in['codigo'] = devol_out['codigo']

devol_in['str_serie'] = devol_out['str_serie']

devol_in['str_numcontrato'] = devol_out['str_numcontrato']

devol_in['dbl_quantidade'] = -devol_out['volta']

In [ ]:
# devol_in['mesa'] = 0
# devol_in['estrategia'] = 0

In [ ]:
map_ajuste

In [ ]:
targets = mapa[mapa['to_lend']>0]
targets = targets.merge(devol_in[['fundo','codigo']].drop_duplicates(),on=['fundo','codigo'],how='inner')
targets = targets[['fundo','mesa','str_estrategia','codigo','to_lend']]

In [ ]:
aux = devol_in.merge(targets[['fundo','mesa','str_estrategia','codigo']],on=['fundo','codigo'],how='inner')



In [ ]:
## Preparar boleta de ajuste doador
aux

In [ ]:
mapa

# 2 volta de ajustes de Internas e Diretas

In [ ]:
map_ajuste = map_ajuste[map_ajuste['to_borrow_0']!=0]

In [ ]:
saldo_total = mapa.groupby(['fundo','codigo']).agg({'custodia_0':sum}).reset_index()
saldo_quebra = mapa.groupby(['fundo','codigo','mesa','str_estrategia']).agg({'custodia_0':sum}).reset_index()

In [ ]:
saldo_quebra = saldo_quebra[saldo_quebra['custodia_0']>0]

In [ ]:
interna_ajuste = map_ajuste.merge(saldo_total,on=['fundo','codigo'],how='left')

In [ ]:
interna_ajuste = interna_ajuste[interna_ajuste['custodia_0']>=0]

In [ ]:
interna_ajuste

In [ ]:
mapa_aux = mapa[mapa['custodia_0']>0].merge(interna_ajuste[['fundo','codigo','mesa','str_estrategia','to_borrow_0']].rename(columns={'to_borrow_0':'qtd target','str_estrategia':'estrategia target','mesa':'mesa target'}),on=['fundo','codigo'],how='inner')

In [ ]:
saldo_quebra = saldo_quebra.merge(interna_ajuste[['fundo','codigo']],on=['fundo','codigo'],how='inner')

In [ ]:
saldo_quebra = saldo_quebra.drop_duplicates()

In [ ]:
saldo_quebra

In [ ]:
mapa_aux =  mapa_aux[['fundo','mesa','str_estrategia','codigo','custodia_0','qtd target','estrategia target','mesa target']]

In [ ]:
mapa_aux

In [ ]:
for i, row in mapa_aux.iterrows():
    saldo =  saldo_quebra.loc[(saldo_quebra['fundo']==row['fundo']) &  (saldo_quebra['codigo']==row['codigo']) &  (saldo_quebra['mesa']==row['mesa']) &  (saldo_quebra['str_estrategia']==row['str_estrategia']),'custodia_0'].item()
    
    saldo_quebra.loc[(saldo_quebra['fundo']==row['fundo']) &  (saldo_quebra['codigo']==row['codigo']) &  (saldo_quebra['mesa']==row['mesa']) &  (saldo_quebra['str_estrategia']==row['str_estrategia']),'custodia_0'] =  saldo - row['qtd target'] 
    


In [ ]:
saldo_quebra['status'] = saldo_quebra['custodia_0'].apply(lambda x: True if x>=0 else False )

In [ ]:
saldo_quebra = saldo_quebra[saldo_quebra['status']==True][['fundo','codigo','mesa','str_estrategia']]

In [ ]:
mapa_aux = mapa_aux.merge(saldo_quebra,on=['fundo','codigo','mesa','str_estrategia'],how='inner')

In [ ]:
mapa_aux['out'] = -mapa_aux['qtd target']

In [ ]:
mapa_aux

In [ ]:
boleta_out = mapa_aux[['fundo','codigo','mesa','str_estrategia','out']]
boleta_in = mapa_aux[['fundo','codigo','mesa target','estrategia target','qtd target']]

In [ ]:
boleta_in.columns = ['str_fundo','str_codigo','str_mesa','str_estrategia','dbl_quantidade']
boleta_out.columns = ['str_fundo','str_codigo','str_mesa','str_estrategia','dbl_quantidade']

In [ ]:
last_interna = pd.concat([boleta_in,boleta_out]).rename(columns = {'str_fundo':'ALOCACAO','str_mesa':'MESA','str_estrategia':'ESTRATEGIA','dbl_quantidade':'NOTIONAL'})

In [ ]:
last_interna['CLEARING'] = 'Interna'
last_interna['CONTRA'] = 'Interna'
last_interna['TIPO'] = 'Emprestimo RV'
last_interna['CODIGO'] = last_interna['str_codigo'].apply(lambda x: x[:4])
last_interna['SIDE'] = last_interna['NOTIONAL'].apply(lambda x: "T" if x>0 else "D")

In [ ]:
## Taxa
def get_taxasalugueis(dt_1,ticker=None):
    if dt_1==None:
        dt = datetime.date.today()
        dt_1 = workdays.workday(dt, -1, holidays_br)
    CORPORATE_DSN_CONNECTION_STRING = "DSN=Kapitalo_Corp"

    # df = pd.read_sql(query, db_conn_risk)
    aux = '"MktNm":"Balcao"'
    connection = pyodbc.connect(CORPORATE_DSN_CONNECTION_STRING)

    df = pd.read_sql(f" CALL up2data.XSP_UP2DATA_DEFAULT('{dt_1.strftime('%Y-%m-%d')}', 'Equities_AssetLoanFileV2', '{aux}' )",connection)
    df.columns = [x.lower() for x in df.columns]
    df =df[df['mktnm']=='Balcao']

    if ticker!=None:
        df = df[df['tckrsymb']==ticker]
    
    return df


In [ ]:
taxas = get_taxasalugueis(dt_1)[['tckrsymb','takravrgrate']].rename(columns={'tckrsymb':'str_codigo','takravrgrate':'taxa'})

In [ ]:
last_interna = last_interna.merge(taxas,on=['str_codigo'],how='inner')

In [ ]:
try:
    last_interna['SERIE'] =last_interna.apply(lambda row: row['str_codigo'] + "-" +row['SIDE']+"-"+ str(row['taxa']).replace('.',',') + "-"+ venc_interna.strftime('%Y%m%d'+"-"+"INTERNO"),axis=1 )

except:
    pass

In [ ]:
last_interna = last_interna[last_interna['NOTIONAL']!=0]

In [ ]:
# last_interna.to_excel('last_interna.xlsx')


In [ ]:
last_interna

In [ ]:
mapa

In [10]:
ctos_btc

,Unnamed: 0,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,codigo,tipo,preco,saldo,taxa,modalidade,vencimento,volume,corretora
0,210,2023-09-25,KAPITALO ZETA MASTER FIM,Emprestimo RV,ABCB,"ABCB4-D-0,14-20231025-BALCAO",Kapitalo 11.1,Bolsa 2,2023091400504897530001-1,-33,ABCB4,D,19.35,-100,0.14,BALCAO,20231025,-4.62,Link
1,211,2023-09-25,KAPITALO ZETA MASTER FIM,Emprestimo RV,ABCB,"ABCB4-D-0,14-20231025-BALCAO",Kapitalo 1.0,Bolsa 2,2023091400504897530001-1,-67,ABCB4,D,19.35,-100,0.14,BALCAO,20231025,-9.38,Link
2,212,2023-09-25,KAPITALO ZETA MASTER FIA,Emprestimo RV,SBSP,"SBSP3-D-0,03-20231016-E1",Kapitalo 11.1,CashCarry,2023091300482693140001-2,-10,SBSP3,D,59.31,-10,0.03,E1,20231016,-0.30,XP
3,213,2023-09-25,KAPITALO ZETA MASTER FIA,Emprestimo RV,SUZB,"SUZB3-D-0,02-20231004-BALCAO",Kapitalo 11.1,CashCarry,2023082400496919070001-1,-1,SUZB3,D,49.45,-723,0.02,BALCAO,20231004,-0.02,XP
4,214,2023-09-25,KAPITALO ZETA MASTER FIA,Emprestimo RV,SUZB,"SUZB3-D-0,02-20231004-BALCAO",Kapitalo 11.1,Bolsa 2,2023082400496919070001-1,-240,SUZB3,D,49.45,-723,0.02,BALCAO,20231004,-4.80,XP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,39627,2023-09-25,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CYRE,"CYRE3-D-0,52-20231106-BALCAO",Kapitalo 11.1,Bolsa 2,2023092500509451580001-1,-1914,CYRE3,D,20.92,-1914,0.52,BALCAO,20231106,-995.28,Terra
5753,39628,2023-09-25,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CYRE,"CYRE3-D-0,52-20231106-BALCAO",Kapitalo 11.1,Bolsa 2,2023092500509451160001-1,-694,CYRE3,D,20.92,-694,0.52,BALCAO,20231106,-360.88,Terra
5754,39629,2023-09-25,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CVCB,"CVCB3-T-4,52-20231002-BALCAO",Kapitalo 11.1,Bolsa 2,2023082200482706750001-2,25406,CVCB3,T,2.51,25406,4.52,BALCAO,20231002,114835.12,Bradesco
5755,39630,2023-09-25,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,CVCB,"CVCB3-T-3,12-20231030-BALCAO",Kapitalo 11.1,Bolsa 2,2023091900507360490001-1,29791,CVCB3,T,2.29,29791,3.12,BALCAO,20231030,92947.92,Bradesco


# Ajustes Gerais

# Ajustes Contratos


In [1]:
print(1)

1


In [2]:

#
import sys

# from devolucao import get_df_devol
sys.path.append("..")
import DB
import workdays
import datetime
import pandas as pd
import numpy as np
import carteira_ibov
import taxas
import os
pd.options.mode.chained_assignment = None  # default='warn'
import config2 as config
import psycopg2
import pandas as pd
import workdays
import pyodbc
#
holidays_br = workdays.load_holidays("BR")
holidays_b3 = workdays.load_holidays("B3")

dt = datetime.date.today()
vcto_0 = dt
dt_pos = workdays.workday(dt, -1, holidays_br)


dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt_3 = workdays.workday(dt, -3, holidays_b3)
dt_4 = workdays.workday(dt, -4, holidays_b3)

dt_next_1 = workdays.workday(dt, 1, holidays_b3)
vcto_1 = dt_next_1
dt_next_2 = workdays.workday(dt, 2, holidays_b3)
vcto_2 = dt_next_2
dt_next_3 = workdays.workday(dt, 3, holidays_b3)
vcto_3 = dt_next_3
dt_next_4 = workdays.workday(dt, 4, holidays_b3)
vcto_4 = dt_next_4
dt_next_5 = workdays.workday(dt, 5, holidays_b3)
vcto_5 = dt_next_5

dt_liq = workdays.workday(dt_1, 4, holidays_b3)

db_conn_test = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user=config.DB_TESTE_USER,
password=config.DB_TESTE_PASS)

query = f"select * from tbl_alugueisconsolidados where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa in ('Kapitalo 11.1','Kapitalo 1.0')"

query_ibotz = f" select str_fundo,str_mesa,str_mercado,str_estrategia,str_codigo,str_serie, str_numcontrato,sum(dbl_quantidade) as lote_ajustado  from ibotz.tbl_boletasalugueis_ibotz where dte_data='{dt.strftime('%Y-%m-%d')}' and str_mesa in ('Kapitalo 11.1','Kapitalo 1.0') and str_mercado in ('Emprestimo RV/AjustePosicao','Emprestimo RV/Devolucao') group by str_fundo,str_mesa,str_mercado,str_estrategia,str_codigo,str_serie, str_numcontrato"



btc_ibotz = pd.read_sql(query_ibotz,db_conn_test)

btc_ibotz['str_serie'] = btc_ibotz['str_serie'].apply(lambda x: x.split('/')[0])
btc_ibotz = btc_ibotz.groupby(['str_fundo','str_mesa','str_mercado','str_estrategia','str_codigo','str_serie', 'str_numcontrato']).sum().reset_index()

ctos_btc = pd.read_sql(query, db_conn_test)
if not btc_ibotz.empty:
    ctos_btc = ctos_btc.merge(btc_ibotz,on=['str_fundo', 'str_mesa', 'str_estrategia', 'str_codigo', 'str_serie',
    'str_numcontrato'],how='outer').fillna(0)
else:
    ctos_btc['lote_ajustado'] = 0
# btc_ibotz.columns = ['index','str_fundo','str_mesa'	,'str_estrategia','clearing','contra','tipo','str_codigo','str_serie','lote_ajustado','dbl_preco','side','str_numcontrato']
# aux_columns = ctos_btc.columns

ctos_btc = ctos_btc.groupby(['str_fundo', 'str_mesa', 'str_estrategia', 'str_codigo', 'str_serie','str_numcontrato']).agg({'dbl_quantidade':sum,'lote_ajustado':sum}).reset_index()
# ctos_btc['preco'] = ctos_btc['str_numcontrato'].map(prices)
ctos_btc['dbl_quantidade_new'] =ctos_btc['dbl_quantidade']+ctos_btc['lote_ajustado']
ctos_btc.to_excel('ctos_ajustados.xlsx')
ctos_btc['tipo'] = ctos_btc['str_serie'].apply(lambda x: x.split('-')[1])
ctos_btc['codigo'] = ctos_btc['str_serie'].apply(lambda x: x.split('-')[0])
ctos_btc['test'] = ctos_btc.apply(lambda row: False if ((row['tipo']=='T')& (row['dbl_quantidade_new']<0) ) else False if ((row['tipo']=='D')& (row['dbl_quantidade_new']>0) ) else True,axis=1 )

ajuste = ctos_btc[ctos_btc['test']==False]
display(ajuste)
target_ajuste = ctos_btc[ctos_btc['test']==True]
target_ajuste['dbl_quantidade_abs'] = target_ajuste['dbl_quantidade_new'].apply(lambda x: abs(x))
target_df = pd.DataFrame()
ajustes_targets = target_ajuste[['str_fundo','str_mesa','str_serie','str_estrategia','str_numcontrato','dbl_quantidade_new']]


ajuste = ajuste[['str_fundo','str_mesa','str_estrategia','codigo','str_serie','tipo','dbl_quantidade_new','str_numcontrato']]

ajuste
# aux = ajuste[ajuste['str_numcontrato']=='2023092000507940640001-1'].copy()
aux = ajuste.copy()
boletas = pd.DataFrame()
if not aux.empty:
    for i,row in aux.iterrows():
        if ajustes_targets.loc[ajustes_targets['str_numcontrato']==row['str_numcontrato']].shape[0]==1:
            
            if (ajustes_targets.loc[ajustes_targets['str_numcontrato']==row['str_numcontrato'],'dbl_quantidade_new'].item() + row['dbl_quantidade_new'])>=0:
                aux_target = ajustes_targets.loc[ajustes_targets['str_numcontrato']==row['str_numcontrato']].copy()
                aux_target['dbl_quantidade_new'] = row['dbl_quantidade_new']
                
                boletas = pd.concat([aux_target,boletas])
                
                boletas = boletas.append({
                    'str_fundo':row['str_fundo'], 
                    'str_mesa':row['str_mesa'],
                     'str_serie':row['str_serie'],
                      'str_estrategia':row['str_estrategia'],
                      'str_numcontrato':row['str_numcontrato'],
                      'dbl_quantidade_new':-row['dbl_quantidade_new']
                },ignore_index=True)
            elif row['tipo']=='D':
                aux_target = ajustes_targets.loc[ajustes_targets['str_numcontrato']==row['str_numcontrato']].copy()
                aux_target['dbl_quantidade_new'] = row['dbl_quantidade_new']
                
                boletas = pd.concat([aux_target,boletas])
                
                boletas = boletas.append({
                    'str_fundo':row['str_fundo'], 
                    'str_mesa':row['str_mesa'],
                     'str_serie':row['str_serie'],
                      'str_estrategia':row['str_estrategia'],
                      'str_numcontrato':row['str_numcontrato'],
                      'dbl_quantidade_new':-row['dbl_quantidade_new']
                },ignore_index=True)
            
        else:
            
            aux_target = ajustes_targets.loc[ajustes_targets['str_numcontrato']==row['str_numcontrato']].copy()
            aux_target['prop'] = aux_target['dbl_quantidade_new']/aux_target['dbl_quantidade_new'].sum()
            aux_target['dbl_quantidade_new'] = (aux_target['prop']*row['dbl_quantidade_new']).round(0)
            boletas = pd.concat([aux_target,boletas])
            boletas = boletas.append({
                'str_fundo':row['str_fundo'], 
                'str_mesa':row['str_mesa'],
                    'str_serie':row['str_serie'],
                    'str_estrategia':row['str_estrategia'],
                    'str_numcontrato':row['str_numcontrato'],
                    'dbl_quantidade_new':-row['dbl_quantidade_new']
            },ignore_index=True).fillna(0)
        
print(boletas)

boletas  = boletas[boletas['dbl_quantidade_new']!=0]

aux_ajuste = boletas.groupby('str_numcontrato').sum().reset_index()


aux_ajuste = aux_ajuste[aux_ajuste['dbl_quantidade_new']!=0]


aux_ajuste = aux_ajuste.set_index('str_numcontrato')['dbl_quantidade_new'].to_dict()


for i,value in aux_ajuste.items():
    aux_max = boletas.loc[boletas['str_numcontrato']==i,'dbl_quantidade_new'].max()
    boletas.loc[(boletas['str_numcontrato']==i)&(boletas['dbl_quantidade_new']==aux_max),'dbl_quantidade_new'] = boletas.loc[(boletas['str_numcontrato']==i)&(boletas['dbl_quantidade_new']==aux_max),'dbl_quantidade_new']  - aux_ajuste[i]


boletas['str_corretora']='Interna'
boletas['str_clearing']='Interna'
boletas['str_mercado']='Emprestimo RV/AjustePosicao'

boletas['str_codigo'] = boletas['str_serie'].apply(lambda x: x[:4])

boletas['dbl_preco'] = 0

boletas = boletas[['str_fundo','str_mesa','str_estrategia','str_clearing','str_corretora','str_mercado','str_codigo','str_serie','dbl_quantidade_new','dbl_preco','str_numcontrato']]

boletas.columns = ['ALOCACAO', 'MESA', 'ESTRATEGIA', 'CLEARING', 'CONTRA', 'TIPO','CODIGO', 'SERIE', 'NOTIONAL', 'PREMIO', 'OBS']

# ibotz_k11.df_to_ibotz_ajuste(boletas)

,str_fundo,str_mesa,str_estrategia,str_codigo,str_serie,str_numcontrato,dbl_quantidade,lote_ajustado,dbl_quantidade_new,tipo,codigo,test


Empty DataFrame
Columns: []
Index: []


KeyError: 'dbl_quantidade_new'

In [43]:
aux_ajuste

{'2023112200520453260001-2': 1.0, '2023112200520456520001-2': 1.0}

In [48]:
import ibotz_k11

<Response [200]>